# Importation des bibliothèques nécessaires

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Définir la graine aléatoire pour la reproductibilité

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)


# Titre : Préparation des données


In [ ]:
# Supposons que nous ayons une fonction pour générer ou charger des permutations
def generate_permutations_data(n_samples, n_permutations):
    X = np.random.permutation(np.arange(n_permutations).reshape(1, -1)).repeat(n_samples, axis=0)
    y = np.median(X, axis=1)
    return X, y


In [ ]:
# Générer des données synthétiques
n_samples = 10000  # Nombre d'échantillons
n_permutations = 10  # Longueur de chaque permutation
X, y = generate_permutations_data(n_samples, n_permutations)

In [ ]:
# Normaliser les données d'entrée
X = X / float(n_permutations - 1)
y = y / float(n_permutations - 1)

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Construction du modèle RNN

In [ ]:
# Initialiser le modèle RNN
model = Sequential()

In [ ]:
# Ajouter la première couche RNN avec 128 unités
model.add(SimpleRNN(128, input_shape=(n_permutations, 1), activation='relu', return_sequences=True))
# Ajouter une couche de dropout pour éviter le surapprentissage
model.add(Dropout(0.2))

In [ ]:
# Ajouter la deuxième couche RNN avec 64 unités
model.add(SimpleRNN(64, activation='relu', return_sequences=True))
# Ajouter une couche de dropout pour éviter le surapprentissage
model.add(Dropout(0.2))

In [ ]:
# Ajouter la troisième couche RNN avec 32 unités
model.add(SimpleRNN(32, activation='relu'))
# Ajouter une couche de dropout pour éviter le surapprentissage
model.add(Dropout(0.2))


In [ ]:
# Ajouter la couche de sortie avec une seule unité (pour la régression)
model.add(Dense(1))

#  Compilation du modèle

In [ ]:
# Compiler le modèle avec l'optimiseur Adam et la fonction de perte de l'erreur quadratique moyenne
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')


# Titre : Entraînement du modèle


In [ ]:
# Entraîner le modèle sur les données d'entraînement
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))


# Titre : Évaluation du modèle

In [ ]:
# Évaluer le modèle sur les données de test
test_loss = model.evaluate(X_test, y_test)
print(f'Perte sur les données de test : {test_loss:.4f}')

In [ ]:
# Faire des prédictions sur les données de test
y_pred = model.predict(X_test)


In [ ]:
# Calculer l'erreur quadratique moyenne entre les prédictions et les valeurs réelles
mse = mean_squared_error(y_test, y_pred)
print(f'Erreur quadratique moyenne sur les données de test : {mse:.4f}')

# Titre : Traçage de la perte d'entraînement et de validation

In [ ]:
# Tracer la perte d'entraînement et de validation au fil des époques
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Perte d\'entraînement')
plt.plot(history.history['val_loss'], label='Perte de validation')
plt.title('Perte d\'entraînement et de validation au fil des époques')
plt.xlabel('Époques')
plt.ylabel('Perte')
plt.legend()
plt.show()

## Le modèle a été entraîné pour prédire la médiane des permutations en utilisant un RNN.
## La performance du modèle peut être encore améliorée en ajustant les hyperparamètres,
## en ajoutant des techniques de régularisation, ou en expérimentant avec différentes architectures de réseau.